# 중간 점검3

final.ipynb
- 도보 경로 동일 - ok
- 교통 수단 열 추가
- 교통 수단 선택
- 무당이 평균 시속 - 17km/h 확인

\+ 추가해야할 사항
- trips.js
    - mapbox 스타일 변경(너무 밝아서 라인이 선명하지 않음)
    - 추가 점검
    - 데이터 설명 추가

# 기본 라이브러리, 함수 세팅

In [55]:
# 라이브러리 불러오기
import numpy as np
import itertools
import requests
import polyline
import json
import os
import math

import random as rd
import pandas as pd
import geopandas as gpd

from datetime import datetime, timedelta

from shapely.geometry import Point

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import warnings 

warnings.filterwarnings('ignore')

In [133]:
# 좌표 거리 생성 함수
## 직선 거리 계산 함수
## 거리 계산할 때 경로를 모르는 경우 사용
def calculate_straight_distance(lat1, lon1, lat2, lon2):
    '''
    좌표 거리 생성 함수 
    - 직선 거리 계산 
    - 경로를 모르는 경우 두 지점 간의 대략적인 직선 거리를 계산할 때 사용

    입력값:
        lat1: 출발지 위도 (float)
        lon1: 출발지 경도 (float)
        lat2: 도착지 위도 (float)
        lon2: 도착지 경도 (float)

    출력값:
        두 지점 간의 직선 거리 (킬로미터 단위, float)
    '''
# 직선 거리 계산
# 입력값: 출발지 위도, 경도 / 도착지 위도, 경도
    # 지구 반경 (킬로미터 단위)
    km_constant = 3959* 1.609344
    
    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    
    # 위도 및 경도 차이 계산
    dlat = lat2 - lat1 # 도착지 위도 - 출발지 위도
    dlon = lon2 - lon1 # 도착지 경도 - 출발지 경도

    # Haversine 공식 계산
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    # 거리 계산 (킬로미터 단위)
    km = km_constant * c
    
    return km # 출발지-도착지 사이의 직선 거리 반환

In [134]:
# trips 데이터 생성 함수
#### osrm 페키지로 경로 추출
# 입력으로 받은 출발지와 목적지 좌표를 이용하여 경로 정보를 가져오는 함수
def get_res(point, mode = 'foot'): # 도보 이용

   # point: 출발지와 목적지 좌표 정보를 포함하는 리스트[lat1, lon1, lat2, lon2]
   # mode: 이동 수단 (기본값: 도보)

   status = 'defined'

   # 요청을 재시도할 수 있도록 세션 객체 생성 및 설정
   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   #### url 생성 코드
   # 전체 경로 정보를 요청
   overview = '?overview=full'
   # lon, lat, lon, lat 형식의 출발지 목적지 좌표
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}"
   # 보행경로 url
   url = f'http://router.project-osrm.org/route/v1/{mode}/'
   # 경로 정보 요청
   r = session.get(url + loc + overview) 
   
   # 만약 경로가 안뜰 때 대체 결과 생성
   ## 직선 거리 구하기
   if r.status_code!= 200:
      
      status = 'undefined'
      
       # 직선 거리 계산
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # 경로 정보 생성 (출발지와 목적지 좌표만 포함)
      route = [[point[0], point[1]], [point[2], point[3]]]

      # 소요 시간 및 타임스탬프 계산 (가정: 보행 속도 10km/h)
      speed_km = 5#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   # 경로 정보를 성공적으로 가져온 경우, JSON 응답을 반환
   res = r.json()   
   return res, status

In [135]:
# 경로를 가는데 걸리는 시간과 거리 추출 함수
def extract_duration_distance(res, speed_kmh):
   # get_res함수에서 추출된 데이터에서 시간과 거리 뽑기
   # 입력값: res(get_res함수에서 추출된 데이터), 속도 (km/h)
   
   distance = res['routes'][0]['distance']
   # JSON 응답에서 첫 번째 경로의 거리 값을 추출
   # m 단위로 거리 추출
   # duration = res['routes'][0]['duration']/(60)  # 분 단위로 변환
   
   # 속도로 시간 계산
   speed_kmh = speed_kmh  # km/h
   speed_mps = speed_kmh * 1000 / 3600  # 속도를 m/s로 변환
   duration = distance / speed_mps / 60  # 분 단위로 변환
   
   return duration, distance # 소요 시간, 거리 반환

# 경로 추출 함수
def extract_route(res):
    # 입력값: res(get_res함수에서 추출된 데이터)
   
    # get_res함수에서 추출된 데이터에서 경로 뽑기
    # 경로가 인코딩 되어 있기 때문에 아래 함수를 써서 디코딩해주어야지 위경도로 이루어진 경로가 나옴
    route = polyline.decode(res['routes'][0]['geometry'])
    
    # 사용할 형식에 맞춰 위경도 좌표의 위치를 바꿔주는 것
    route = list(map(lambda data: [data[1],data[0]] ,route))
    # data: [위도, 경도] 형식의 좌표 쌍 -> [경도, 위도] 형식의 좌표 쌍

    return route # [[127.0, 37.0], [127.1, 37.1], ...] 형식의 경로 반환

In [136]:
# 총 걸리는 시간을 경로의 거리 기준으로 쪼개주는 함수
def extract_timestamp(route, duration):
    '''
    경로의 각 구간 거리 비율에 따라 예상 도착 시간을 계산하는 함수

    입력값:
        route: 경로 정보 (위도와 경도로 구성된 리스트)
        duration: 총 소요 시간 (분 단위, float)
    
    출력값:
        timestamp: 각 지점의 예상 도착 시간 리스트 (분 단위)
    '''
    # 입력값: route(경로 정보), duration(총 소요 시간)
    
    # 리스트를 numpy이 배열로 변경
    rt = np.array(route)
    # 리스트를 수평 기준으로 합치기
    rt = np.hstack([rt[:-1,:], rt[1:,:]])
    # [출발점_lat, 출발점_lon, 도착점_lat, 도착점_lon] 형식으로 변환

    # 각각 직선거리 추출(리스트 형태)
    per = calculate_straight_distance(rt[:,1], rt[:,0], rt[:,3], rt[:,2])
    # 출발점 경도, 출발점 위도, 도착점 경도, 도착점 위도를 이용하여 직선거리 계산

    # 각각의 직선거리를 전체 직선거리의 합으로 나누기
    per = per / np.sum(per)
    # 각 구간의 직선 거리 비율 계산
    # 전체 경로에서 해당 구간이 차지하는 비율 계산

    # 계산된 비율을 기반으로 각 지점 도착 예상 시간 계산
    timestamp = per * duration
    ## 각 구간의 비율에 전체 소요 시간을 곱해 각 구간의 소요 시간 계산
    
    # 각 구간의 소요 시간을 누적하여 더하기
    timestamp = np.hstack([np.array([0]),timestamp])

    # 타임스탬프 배열의 누적 합을 계산하여 각 지점의 예상 도착 시간을 생성
    timestamp = list(itertools.accumulate(timestamp)) 
    
    return timestamp # 각 지점의 예상 도착 시간 리스트 e.g. [0, 10, 20, 30, ...] 반환

In [137]:
from functools import partial

# 모든 함수를 한번에 실행하는 코드(trips 데이터의 형태로 저장)
def osrm_routing_machine(O, D, mode, speed_kmh):
   # 입력값: 출발지 좌표, 목적지 좌표, 이동 수단, 속도

   # osrm 데이터 생성
   osrm_base, status = get_res([O.x, O.y, D.x, D.y], mode)
   
   # osrm 데이터가 생성 됬으면 진행
   if status == 'defined':
      # 거리 및 걸리는 시간 추출
      duration, distance = extract_duration_distance(osrm_base, speed_kmh)
      # 경로 추출
      route = extract_route(osrm_base)
      # timestamp 생성
      timestamp = extract_timestamp(route, duration)
      # 결과 저장
      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
      
      return result
   else: 
      return osrm_base # 경로 데이터 없을 때는 직선거리 데이터 반환
   
# OD_data 한쌍일 때 osrm_routing_machine작동함수
def osrm_routing_machine_multiprocess(OD, mode, speed_kmh):
   O, D = OD
   result = osrm_routing_machine(O, D, mode, speed_kmh)
   return result
# OD_data 데이터가 리스트쌍 일때의 osrm_routing_machine 작동함수
def osrm_routing_machine_multiprocess_all(OD_data, mode, speed_kmh):
    results = list(map(partial(osrm_routing_machine_multiprocess, mode = mode, speed_kmh=speed_kmh), OD_data))
    return results

In [138]:
# 출발시간 기반으로 탑승시간 계산 함수
def calculate_boarding_time(start_times):
    '''
    출발시간을 기반으로 탑승 시간을 계산하는 함수
    - 버스는 10분 간격으로 출발한다고 가정

    입력값:
        start_times: 출발 시간 리스트 (분 단위)

    출력값:
        boarding_times: 각 출발 시간에 대응하는 탑승 시간 리스트 (분 단위)
    '''
    boarding_times = []
    for start_time in start_times:
        # 버스는 10분 간격으로 출발한다고 가정
        boarding_time = math.ceil(start_time / 10) * 10
        boarding_times.append(boarding_time)
    return boarding_times

# 데이터프레임에서 OD 데이터와 출발시간, 탑승시간 데이터 추출 함수
def extract_od_and_start_time(df):
    '''
    데이터프레임에서 OD 데이터와 출발 시간, 탑승 시간 데이터를 추출하는 함수

    입력값:
        df: OD 데이터가 포함된 데이터프레임
            - 컬럼: start_point (출발지 위경도), end_point (도착지 위경도), start_time, boarding_time

    출력값:
        od_data: OD 데이터 리스트 ([[출발지 위경도, 도착지 위경도], ...])
        start_time: 출발 시간 리스트
        boarding_time: 탑승 시간 리스트
    '''
    # OD 데이터를 추출 (출발점과 도착점의 위경도)
    od_data = [[row['start_point'], row['end_point']] for _, row in df.iterrows()]
    
    # 출발시간과 탑승시간 데이터를 리스트로 추출
    start_time = df['start_time'].tolist()
    boarding_time = df['boarding_time'].tolist()
    
    return od_data, start_time, boarding_time

# 무당이 데이터 불러오기

In [62]:
path = 'final_simulation/data_final/'

# Load the JSON file
with open(os.path.join(path + 'stops_point.json'), "r", encoding="utf-8") as json_file: 
    stops_point = json.load(json_file) # 무당이가 정차하는 정류장들의 좌표

with open(os.path.join(path + 'station_schedule.json'), "r", encoding="utf-8") as json_file: 
    station_schedule = json.load(json_file) # 정류장마다 무당이가 출발하는 시간

    # e.g. 600: 60으로 나누면 10 => 10시

In [63]:
stops_point

{'가천대_ai공학관': [127.133374, 37.455009],
 '가천대_총학생회': [127.133923, 37.453348],
 '가천대_일반대학원': [127.130112, 37.452589],
 '가천대_반도체대학': [127.127384, 37.45091],
 '가천대_학생회관': [127.130121, 37.450268],
 '가천대_교육대학원': [127.131698, 37.452066]}

In [64]:
# 각 정류장에서의 무당이 출발 시간
station_schedule['가천대_ai공학관']
## 10시부터 ~ 18시까지 10분 간격으로 출발

[600.0,
 610.0,
 620.0,
 630.0,
 640.0,
 650.0,
 660.0,
 670.0,
 680.0,
 690.0,
 700.0,
 710.0,
 720.0,
 730.0,
 740.0,
 750.0,
 760.0,
 770.0,
 780.0,
 790.0,
 800.0,
 810.0,
 820.0,
 830.0,
 840.0,
 850.0,
 860.0,
 870.0,
 880.0,
 890.0,
 900.0,
 910.0,
 920.0,
 930.0,
 940.0,
 950.0,
 960.0,
 970.0,
 980.0,
 990.0,
 1000.0,
 1010.0,
 1020.0,
 1030.0,
 1040.0,
 1050.0,
 1060.0,
 1070.0,
 1080.0]

In [65]:
from shapely.geometry import Point

path = 'final_simulation/data_final/OD_data_ladybug' # 학생 300명의 가상의 이동데이터

# GeoDataFrame으로 Shapefile 읽기
OD_data_ladybug = gpd.read_file(path, encoding = 'euckr')

# 컬러명 수정
OD_data_ladybug.rename(columns={
    'start_time': 'start_time',
    'start_poin': 'start_point',
    'start_stat': 'start_station',
    'end_point': 'end_point',
    'end_statio': 'end_station',
    'ladybug_pa': 'ladybug_path'
}, inplace=True)

# 문자열을 Point 객체로 변환하는 함수
def convert_to_point(point_str):
    # 'POINT (127.127384 37.45091)' -> [127.127384, 37.45091]
    coords = point_str.replace("POINT (", "").replace(")", "").split()
    return Point(float(coords[0]), float(coords[1]))

# 'start_point'와 'end_point'를 Point 객체로 변환
OD_data_ladybug['start_point'] = OD_data_ladybug['start_point'].apply(convert_to_point)
OD_data_ladybug['end_point'] = OD_data_ladybug['end_point'].apply(convert_to_point)

OD_data_ladybug # 한 행당 한 명의 학생의 이동 데이터

,start_time,start_point,start_station,end_point,end_station,ladybug_path,geometry
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관,가천대_반도체대학 -> 가천대_학생회관,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학,가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대학,"LINESTRING (127.13390 37.45337, 127.13375 37.4..."
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학...,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
...,...,...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원,가천대_교육대학원 -> 가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원,"LINESTRING (127.13181 37.45222, 127.13176 37.4..."
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관,가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13001 37.45030, 127.13007 37.4..."
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관,가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13181 37.45222, 127.13176 37.4..."
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원,가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대...,"LINESTRING (127.13322 37.45500, 127.13325 37.4..."


# 무당이 이용할 경우

In [139]:
# 대기 시간 및 탑승 시간 계산 함수
def calculate_waiting_and_boarding_time(df, station_schedule):
    '''
    각 출발 시간에 대한 대기 시간과 탑승 시간을 계산하는 함수
    - 다음 무당이 출발 시간 - 현재 출발 시간 = 대기 시간

    입력값:
        df: OD 데이터가 포함된 데이터프레임
            - 컬럼: start_time (출발 시간), start_station (출발 정류장)
        station_schedule: 각 정류장의 무당이 스케줄 (dict 형식, {정류장명: [시간 리스트]})

    출력값:
        df: 대기 시간(waiting_time)과 탑승 시간(boarding_time)이 추가된 데이터프레임
    '''
    waiting_times = [] # 대기 시간 저장
    for index, row in df.iterrows(): # 데이터프레임을 행 단위로 반복해서 받아옴
        start_time_minutes = row["start_time"] # 출발 시간 (분 단위)
        schedule = station_schedule[row["start_station"]]  # 정류장의 스케줄 불러오기(행 단위)
        
        # 현재 시간 이후의 버스 시간 계산
        next_ladybug_time = min([time for time in schedule if time >= start_time_minutes], default=None)
        ## 현재 시간 이후 버스 시간 중 가장 빠른 시간을 찾음

        if next_ladybug_time is None:  # 같은 시간 내에 버스가 없으면 다음 시간의 첫 버스 사용
            next_ladybug_time = schedule[0]
            waiting_time = (1440 - start_time_minutes) + next_ladybug_time # 1440: 24시간
        else:
            waiting_time = next_ladybug_time - start_time_minutes
        
        waiting_times.append(waiting_time)
    
    # 대기 시간 및 탑승 시간 계산 결과를 데이터프레임에 추가
    df["waiting_time"] = waiting_times
    df["boarding_time"] = df["start_time"] + df["waiting_time"]  # 탑승 시간은 원래 시간 단위로 추가
    return df

In [140]:
# 함수 적용
updated_OD_data_ladybug = calculate_waiting_and_boarding_time(OD_data_ladybug, station_schedule)

# 결과 출력
updated_OD_data_ladybug # 대기시간과 탑승시간이 추가된 데이터

,start_time,start_point,start_station,end_point,end_station,ladybug_path,geometry,waiting_time,boarding_time,total_duration,segment_duration,total_distance_m,arrival_time
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관,가천대_반도체대학 -> 가천대_학생회관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.950000,665.0,5.001765,1.051765,298.0,666.051765
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학,가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대학,"LINESTRING (127.13390 37.45337, 127.13375 37.4...",2.216667,681.3,4.960431,2.743765,777.4,684.043765
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학...,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.650000,645.0,9.550118,5.900118,1671.7,650.900118
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",2.850000,635.0,6.530824,3.680824,1042.9,638.680824
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",6.650000,625.0,8.759176,2.109176,597.6,627.109176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원,가천대_교육대학원 -> 가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",8.650000,1008.1,12.440941,3.790941,1074.1,1011.890941
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관,가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13001 37.45030, 127.13007 37.4...",7.666667,996.6,10.295725,2.629059,744.9,999.229059
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관,가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",5.750000,1008.1,7.321647,1.571647,445.3,1009.671647
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원,가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대...,"LINESTRING (127.13322 37.45500, 127.13325 37.4...",5.916667,1030.0,11.490667,5.574000,1579.3,1035.574000


In [68]:
# 정류장간의 경로와 시간
path = 'final_simulation/data_final/distance_by_stops_geo' 

# GeoDataFrame으로 Shapefile 읽기
distance_by_stops_geo = gpd.read_file(path, encoding = 'euckr')

distance_by_stops_geo

,start,end,distance_m,time,geometry
0,가천대_ai공학관,가천대_총학생회,204.3,0.721059,"LINESTRING (127.13322 37.45500, 127.13325 37.4..."
1,가천대_총학생회,가천대_일반대학원,424.5,1.498235,"LINESTRING (127.13390 37.45337, 127.13375 37.4..."
2,가천대_일반대학원,가천대_반도체대학,352.9,1.245529,"LINESTRING (127.13017 37.45249, 127.13004 37.4..."
3,가천대_반도체대학,가천대_학생회관,298.0,1.051765,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
4,가천대_학생회관,가천대_교육대학원,299.6,1.057412,"LINESTRING (127.13001 37.45030, 127.13007 37.4..."
5,가천대_교육대학원,가천대_ai공학관,445.3,1.571647,"LINESTRING (127.13181 37.45222, 127.13176 37.4..."


In [141]:
def calculate_total_duration_and_distance_m(df, distance_by_stops_geo):
    total_durations = []  # 전체 걸린 시간 저장 (대기 시간 포함)
    arrival_times = []    # 도착 시간 저장
    segment_durations = []  # 순수 구간별 소요 시간 저장 (대기 시간 제외)
    total_distances = []  # 총 이동 거리 저장

    for index, row in df.iterrows():
        ladybug_path = row["ladybug_path"]  # "->"로 연결된 구간
        boarding_time = row["boarding_time"]
        start_time = row["start_time"]
        waiting_time = row["waiting_time"]

        # ladybug_path를 구간 리스트로 변환
        segments = [segment.strip() for segment in ladybug_path.split("->")]  # 정류장 별로 분류

        # 구간별 소요 시간과 거리 계산
        total_duration = waiting_time
        segment_duration = 0  # 순수 구간별 소요 시간 초기화
        total_distance_m = 0  # 총 이동 거리 초기화
        for i in range(len(segments) - 1):
            start_segment = segments[i]
            end_segment = segments[i + 1]

            # 각 구간에 대한 소요 시간 및 거리 가져오기
            distance_row = distance_by_stops_geo[
                ((distance_by_stops_geo["start"] == start_segment) & (distance_by_stops_geo["end"] == end_segment)) |
                ((distance_by_stops_geo["start"] == end_segment) & (distance_by_stops_geo["end"] == start_segment))
            ]  # 출발지 도착지 바뀐 경우도 고려

            if not distance_row.empty:
                travel_time = distance_row["time"].values[0]  # 분 단위를 600 단위로 변환
                travel_distance = distance_row["distance_m"].values[0]  # 거리 가져오기
                total_duration += travel_time
                segment_duration += travel_time  # 순수 구간 소요 시간 추가
                total_distance_m += travel_distance  # 총 이동 거리 추가
            else:
                travel_time = 0  # 구간이 없으면 0으로 처리
                travel_distance = 0

        # 도착 시간 계산
        arrival_time = start_time + total_duration

        # 결과 저장
        total_durations.append(total_duration)
        segment_durations.append(segment_duration)
        arrival_times.append(arrival_time)
        total_distances.append(total_distance_m)

    # 데이터프레임에 열 추가
    df["total_duration"] = total_durations
    df["segment_duration"] = segment_durations
    df["arrival_time"] = arrival_times
    df["total_distance_m"] = total_distances

    return df

In [142]:
ladybug = calculate_total_duration_and_distance_m(updated_OD_data_ladybug, distance_by_stops_geo)
ladybug # 총 이동시간, 도착 시간, 총 이동거리가 추가된 데이터

,start_time,start_point,start_station,end_point,end_station,ladybug_path,geometry,waiting_time,boarding_time,total_duration,segment_duration,total_distance_m,arrival_time
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관,가천대_반도체대학 -> 가천대_학생회관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.950000,665.0,5.001765,1.051765,298.0,666.051765
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학,가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대학,"LINESTRING (127.13390 37.45337, 127.13375 37.4...",2.216667,681.3,4.960431,2.743765,777.4,684.043765
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학...,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.650000,645.0,9.550118,5.900118,1671.7,650.900118
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",2.850000,635.0,6.530824,3.680824,1042.9,638.680824
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",6.650000,625.0,8.759176,2.109176,597.6,627.109176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원,가천대_교육대학원 -> 가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",8.650000,1008.1,12.440941,3.790941,1074.1,1011.890941
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관,가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13001 37.45030, 127.13007 37.4...",7.666667,996.6,10.295725,2.629059,744.9,999.229059
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관,가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",5.750000,1008.1,7.321647,1.571647,445.3,1009.671647
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원,가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대...,"LINESTRING (127.13322 37.45500, 127.13325 37.4...",5.916667,1030.0,11.490667,5.574000,1579.3,1035.574000


In [143]:
# 사용할 수 있는 형태로 변경
OD_data, start_time, boarding_time = extract_od_and_start_time(ladybug)

In [144]:
from shapely.geometry import LineString  # GIS 데이터를 처리하기 위한 shapely 모듈
from shapely.ops import nearest_points  # 가장 가까운 점을 계산하기 위한 모듈
def ladybug_path_geometry(geometry, ladybug_path, speed_kmh, total_distance_m):
    """
    ladybug_path와 geometry를 기반으로 경로 정보 및 각 구간의 시간, 거리 정보를 생성하는 함수.

    입력값:
        geometry: 이동 경로를 나타내는 LineString 객체
        ladybug_path: 정류장 경로 ("정류장1->정류장2->..." 형식의 문자열)
        speed_kmh: 이동 속도 (km/h 단위)
        total_distance_m: 전체 이동 거리 (m 단위)

    출력값:
        result: {
            "route": 경로의 좌표 리스트,
            "timestamp": 각 경로 구간의 누적 소요 시간 (분 단위),
            "duration": 전체 소요 시간 (분 단위),
            "distance": 전체 이동 거리 (m 단위)
        }
    """
    if not isinstance(geometry, LineString):
        raise ValueError("geometry는 LineString이어야 합니다.")

    # ladybug_path를 정류장 리스트로 변환
    stations_in_path = [station.strip() for station in ladybug_path.split("->")]

    # geometry의 모든 좌표를 리스트로 변환
    line_coords = list(geometry.coords)

    # 정류장 좌표 매핑
    station_coordinates = []
    for station in stations_in_path:
        station_point = Point(line_coords[0])  # 초기값 설정
        nearest_point = nearest_points(geometry, station_point)[0]
        station_coordinates.append({"station": station, "coord": nearest_point.coords[0]})

    # 최종 경로 생성 (기존 좌표에 정류장 좌표 추가)
    final_route = line_coords
    for station in station_coordinates:
        if station["coord"] not in final_route:
            final_route.append(station["coord"])

    # 경로를 LineString으로 변환
    final_line = LineString(final_route)

    # 경로의 모든 좌표를 numpy 배열로 변환
    route = np.array(list(final_line.coords))

    # 각 구간의 거리 계산
    rt = np.hstack([route[:-1, :], route[1:, :]])  # [출발_lat, 출발_lon, 도착_lat, 도착_lon] 형태로 변환
    distances = calculate_straight_distance(rt[:, 1], rt[:, 0], rt[:, 3], rt[:, 2])

    # 각 구간의 거리 비율 계산
    distance_ratios = distances / np.sum(distances) if np.sum(distances) > 0 else np.zeros_like(distances)

    # 전체 이동 시간 계산
    speed_mps = speed_kmh * 1000 / 3600  # 속도를 m/s로 변환
    segment_duration = total_distance_m / speed_mps / 60  # 전체 소요 시간 (분 단위)

    # 비율 기반으로 각 구간의 소요 시간 계산
    segment_times = distance_ratios * segment_duration

    # 누적 시간 계산
    timestamps = np.hstack([0, np.cumsum(segment_times)]).tolist()

    # 결과 반환
    result = {
        "route": list(final_route),
        "timestamp": timestamps,
        "duration": segment_duration,  # 전체 소요 시간
        "distance": total_distance_m,  # 전체 거리
    }
    return result


def osrm_routing_machine_multiprocess_all_ladybug(df, mode, speed_kmh):
    """
    데이터프레임의 모든 행에 대해 ladybug_path_geometry 함수를 호출하여 경로 정보를 생성.

    입력값:
        df: 입력 데이터프레임 (각 행에 geometry, ladybug_path, total_distance_m 포함)
        mode: 이동 모드 (예: "foot", "car" 등)
        speed_kmh: 이동 속도 (km/h 단위)

    출력값:
        results: 각 행에 대한 경로 정보 리스트
    """
    results = []
    for _, row in df.iterrows():
        geometry = row["geometry"]  # 이동 경로를 나타내는 geometry (LineString)
        ladybug_path = row["ladybug_path"]  # 정류장 경로
        total_distance_m = row["total_distance_m"]  # 전체 이동 거리 (m 단위)

        # ladybug_path_geometry 호출
        result = ladybug_path_geometry(
            geometry, ladybug_path, speed_kmh, total_distance_m
        )
        results.append(result)
    return results

In [145]:
OD_results_ladybug = osrm_routing_machine_multiprocess_all_ladybug(ladybug, 'car', 17)
# 이동 거리 등을 알아야 하기 때문에 OD_data가 아닌 ladybug 데이터 사용

In [146]:
def update_timestamps_route(OD_results, start_times, boarding_times):
    """
    트립 데이터에서 출발 시간을 무당이 출발 시간으로 변경하여 타임스탬프를 업데이트하는 함수

    입력값:
        OD_results: 생성된 트립 데이터 리스트
            - 각 항목은 'timestamp'와 'route'를 포함하는 딕셔너리
        start_times: 각 트립의 출발 시간 리스트
        boarding_times: 각 트립의 무당이 탑승 시간 리스트

    출력값:
        updated_results: 타임스탬프와 경로가 업데이트된 트립 데이터 리스트
    """
    updated_results = []  # 업데이트된 결과를 저장할 리스트

    # OD_results와 start_times, boarding_times를 순회
    for result, start_time, boarding_time in zip(OD_results, start_times, boarding_times):
        # 기존 타임스탬프를 boarding_time으로 조정하고 start_time을 맨 앞에 추가
        updated_timestamps = [start_time] + [t + boarding_time for t in result['timestamp']]

        # 기존 결과를 복사하여 timestamp를 업데이트
        updated_result = result.copy()
        updated_result['timestamp'] = updated_timestamps

        # route의 첫 번째 항목을 복제하여 맨 앞에 추가
        updated_result['route'].insert(0, updated_result['route'][0])

        # 업데이트된 결과를 리스트에 추가
        updated_results.append(updated_result)

    return updated_results

In [147]:
updated_OD_results_ladybug = update_timestamps_route(OD_results_ladybug, start_time, boarding_time)

In [148]:
updated_OD_results_ladybug

[{'route': [(127.12748, 37.45091),
   (127.12748, 37.45091),
   (127.12749, 37.45065),
   (127.12749, 37.45052),
   (127.1275, 37.45052),
   (127.12754, 37.45051),
   (127.12782, 37.45042),
   (127.12786, 37.45042),
   (127.12798, 37.45041),
   (127.12914, 37.4501),
   (127.12916, 37.45009),
   (127.12952, 37.44999),
   (127.12956, 37.44998),
   (127.12964, 37.44996),
   (127.12966, 37.44997),
   (127.12974, 37.44997),
   (127.12982, 37.45),
   (127.12988, 37.45005),
   (127.12993, 37.45013),
   (127.13001, 37.4503)],
  'timestamp': [661.05,
   665.0,
   665.1019255911431,
   665.152864646846,
   665.1559753725793,
   665.1690206639161,
   665.262989448673,
   665.2754323682486,
   665.312966222137,
   665.6937081599857,
   665.7010607561081,
   665.8197048905521,
   665.8327502659795,
   665.8588410215933,
   665.8661936268725,
   665.8910796157386,
   665.9186022634683,
   665.9456615876597,
   665.9806552946253,
   666.0517647058823],
  'duration': 1.051764705882353,
  'distance': 2

In [ ]:
# ScatterplotLayer에 필요한 데이터 생성 함수
def create_scatterplot_data(data):
    """
    시뮬레이션에서 대기 중인 사람의 위치와 시간 정보를 포함한 포인트 데이터를 생성하는 함수

    입력값:
        data: OD 데이터 리스트
            - 각 항목은 'route'와 'timestamp' 키를 포함해야 함

    출력값:
        scatterplot_data: 대기 및 이동 정보를 담은 포인트 데이터 리스트
            - 각 항목은 {
                "coordinates": [경도, 위도],  # 대기 위치
                "timestamp": [시작 시간, 종료 시간]  # 대기 시간 범위
              }
    """
    scatterplot_data = []  # 결과를 저장할 리스트

    # 입력 데이터를 순회하며 Scatterplot 데이터 생성
    for item in data:
        if "route" in item and "timestamp" in item:
            # 경로의 첫 번째 좌표 (대기 위치)
            start_point = item["route"][0]  # 첫 번째 좌표
            # 타임스탬프 시작 및 종료 시간
            start_time = item["timestamp"][0]  # 대기 시작 시간
            end_time = item["timestamp"][1] if len(item["timestamp"]) > 1 else start_time  # 대기 종료 시간

            # 포인트 데이터를 리스트에 추가
            scatterplot_data.append({
                "coordinates": start_point,  # [경도, 위도]
                "timestamp": [start_time, end_time],  # 대기 시간 범위
            })

    return scatterplot_data  # ScatterplotLayer 데이터 반환


# 데이터 생성
scatterplot_data = create_scatterplot_data(updated_OD_results_ladybug)

In [113]:
# icon data 설정
icon_data = [{"name": name, "coordinates": coordinates} for name, coordinates in stops_point.items()]
icon_data

[{'name': '가천대_ai공학관', 'coordinates': [127.133374, 37.455009]},
 {'name': '가천대_총학생회', 'coordinates': [127.133923, 37.453348]},
 {'name': '가천대_일반대학원', 'coordinates': [127.130112, 37.452589]},
 {'name': '가천대_반도체대학', 'coordinates': [127.127384, 37.45091]},
 {'name': '가천대_학생회관', 'coordinates': [127.130121, 37.450268]},
 {'name': '가천대_교육대학원', 'coordinates': [127.131698, 37.452066]}]

# 도보를 이용하는 경우

In [149]:
# 도보 이용 시 시간 계산
OD_results_foot = osrm_routing_machine_multiprocess_all(OD_data, 'foot', 5)
# 출발 - 도착 위치만 알면 됨으로 OD_data 사용

In [150]:
# 생성된 트립 데이터에 출발시간을 변경
def update_timestamps_with_start_time(OD_results, start_times):
    """
    도보 이동 트립 데이터에서 출발 시간을 반영하여 타임스탬프를 갱신하는 함수

    입력값:
        OD_results: 트립 데이터 리스트
            - 각 항목은 'timestamp' 키를 포함하는 딕셔너리
        start_times: 각 트립의 출발 시간 리스트 (분 단위)

    출력값:
        updated_results: 출발 시간이 반영된 트립 데이터 리스트
    """
    # OD_results와 start_times를 순회하며 타임스탬프 갱신
    updated_results = []
    for result, start_time in zip(OD_results, start_times):
        # 기존 timestamp를 start_time과 합산
        updated_timestamps = [t + start_time for t in result['timestamp']]
        # 기존 결과를 복사하고 timestamp를 업데이트
        updated_result = result.copy()
        updated_result['timestamp'] = updated_timestamps
        updated_results.append(updated_result)
    return updated_results

In [151]:
# 도보 이동 트립 데이터의 출발 시간 갱신
updated_OD_results_foot = update_timestamps_with_start_time(OD_results_foot, start_time)

In [152]:
updated_OD_results_foot

[{'route': [[127.12748, 37.45091],
   [127.12749, 37.45065],
   [127.12749, 37.45052],
   [127.1275, 37.45052],
   [127.12754, 37.45051],
   [127.12782, 37.45042],
   [127.12786, 37.45042],
   [127.12798, 37.45041],
   [127.12914, 37.4501],
   [127.12916, 37.45009],
   [127.12952, 37.44999],
   [127.12956, 37.44998],
   [127.12964, 37.44996],
   [127.12966, 37.44997],
   [127.12974, 37.44997],
   [127.12982, 37.45],
   [127.12988, 37.45005],
   [127.12993, 37.45013],
   [127.13001, 37.4503]],
  'timestamp': [661.05,
   661.3965470098866,
   661.569739799276,
   661.5803162667695,
   661.6246702573144,
   661.9441641254882,
   661.9864700520452,
   662.1140851552656,
   663.4086077439515,
   663.4336065707672,
   663.8369966278768,
   663.8813509043301,
   663.9700594734173,
   663.9950583313664,
   664.0796706935112,
   664.173247695792,
   664.2652493980432,
   664.3842280017261,
   664.626],
  'duration': 3.576,
  'distance': 298},
 {'route': [[127.1339, 37.45337],
   [127.13375, 37.

# 도보 이용하는 경우 데이터를 전체 데이터에 적용하기

In [153]:
# ladybug 데이터에 updated_OD_results_foot으로 나온 foot_duration과 foot_distance를 추가

# 기존 데이터프레임 ladybug에 foot_duration과 foot_distance 열 추가
# updated_OD_results_foot에 있는 duration과 distance 값을 추가

# ladybug 데이터에 updated_OD_results_foot에서 추출한 foot_duration과 foot_distance 추가

# foot_duration, foot_distance 데이터 리스트 생성
total_OD_data = ladybug.copy()
foot_durations = [entry['duration'] for entry in updated_OD_results_foot]
foot_distances = [entry['distance'] for entry in updated_OD_results_foot]

# 길이가 다를 경우 데이터 정합성 체크
if len(ladybug) == len(foot_durations):
    # foot_duration과 foot_distance 열 추가
    total_OD_data['foot_duration'] = foot_durations
    total_OD_data['foot_distance'] = foot_distances
else:
    # 길이가 다르면 오류 메시지 출력
    print(f"Error: total_OD_data({len(total_OD_data)})와 updated_OD_results_foot({len(foot_durations)})의 길이가 일치하지 않습니다. 확인이 필요합니다.")

In [154]:
total_OD_data

,start_time,start_point,start_station,end_point,end_station,ladybug_path,geometry,waiting_time,boarding_time,total_duration,segment_duration,total_distance_m,arrival_time,foot_duration,foot_distance
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관,가천대_반도체대학 -> 가천대_학생회관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.950000,665.0,5.001765,1.051765,298.0,666.051765,3.5760,298.0
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학,가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대학,"LINESTRING (127.13390 37.45337, 127.13375 37.4...",2.216667,681.3,4.960431,2.743765,777.4,684.043765,9.3276,777.3
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학...,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.650000,645.0,9.550118,5.900118,1671.7,650.900118,4.2348,352.9
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",2.850000,635.0,6.530824,3.680824,1042.9,638.680824,11.7792,981.6
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",6.650000,625.0,8.759176,2.109176,597.6,627.109176,6.7656,563.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원,가천대_교육대학원 -> 가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",8.650000,1008.1,12.440941,3.790941,1074.1,1011.890941,2.5308,210.9
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관,가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13001 37.45030, 127.13007 37.4...",7.666667,996.6,10.295725,2.629059,744.9,999.229059,8.6088,717.4
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관,가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",5.750000,1008.1,7.321647,1.571647,445.3,1009.671647,5.3436,445.3
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원,가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대...,"LINESTRING (127.13322 37.45500, 127.13325 37.4...",5.916667,1030.0,11.490667,5.574000,1579.3,1035.574000,5.3436,445.3


In [155]:
# 교통 수단 선택
# 교통 수단 열 추가: total_duration과 foot_duration 비교
def determine_transport_mode(row):
    if row['total_duration'] < row['foot_duration']:
        return "ladybug"
    else:
        return "foot"

# apply를 사용해 새로운 열 추가
total_OD_data['교통수단'] = total_OD_data.apply(determine_transport_mode, axis=1)

In [156]:
total_OD_data

,start_time,start_point,start_station,end_point,end_station,ladybug_path,geometry,waiting_time,boarding_time,total_duration,segment_duration,total_distance_m,arrival_time,foot_duration,foot_distance,교통수단
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관,가천대_반도체대학 -> 가천대_학생회관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.950000,665.0,5.001765,1.051765,298.0,666.051765,3.5760,298.0,foot
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학,가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대학,"LINESTRING (127.13390 37.45337, 127.13375 37.4...",2.216667,681.3,4.960431,2.743765,777.4,684.043765,9.3276,777.3,ladybug
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학...,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",3.650000,645.0,9.550118,5.900118,1671.7,650.900118,4.2348,352.9,foot
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",2.850000,635.0,6.530824,3.680824,1042.9,638.680824,11.7792,981.6,ladybug
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원,"LINESTRING (127.12748 37.45091, 127.12749 37.4...",6.650000,625.0,8.759176,2.109176,597.6,627.109176,6.7656,563.8,foot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원,가천대_교육대학원 -> 가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",8.650000,1008.1,12.440941,3.790941,1074.1,1011.890941,2.5308,210.9,foot
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관,가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13001 37.45030, 127.13007 37.4...",7.666667,996.6,10.295725,2.629059,744.9,999.229059,8.6088,717.4,foot
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관,가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13181 37.45222, 127.13176 37.4...",5.750000,1008.1,7.321647,1.571647,445.3,1009.671647,5.3436,445.3,foot
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원,가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대...,"LINESTRING (127.13322 37.45500, 127.13325 37.4...",5.916667,1030.0,11.490667,5.574000,1579.3,1035.574000,5.3436,445.3,foot


In [157]:
# 교통 수단의 개수 계산
transport_mode_counts = total_OD_data['교통수단'].value_counts()

# 결과 출력
print(transport_mode_counts)

교통수단
foot       222
ladybug     78
Name: count, dtype: int64


# 최종 데이터 저장

In [158]:
###### 최대 시간을 봐서 시뮬레이션의 min, max 시간에 활용( 안 짤리도록 )
###### trip.js에 결과 반영 (600  ~ 1095로 설정)
all_timestamps = [t for item in updated_OD_results_ladybug + updated_OD_results_foot for t in item['timestamp']]
# 최대값 계산
max_timestamp = max(all_timestamps)
min_timestamp = min(all_timestamps)

min_timestamp, max_timestamp

(600.6166666666667, 1094.1882352941175)

In [159]:
# 데이터 저장
path = 'final_simulation/public/data/'

with open(os.path.join(path + 'trips_foot.json'), 'w', encoding='utf-8') as file:
    json.dump(updated_OD_results_foot, file)
    
with open(os.path.join(path + 'trips_car.json'), 'w', encoding='utf-8') as file:
    json.dump(updated_OD_results_ladybug, file)
    
with open(os.path.join(path + 'icon_data.json'), 'w', encoding='utf-8') as file:
    json.dump(icon_data, file)
    
with open(os.path.join(path + 'trips_car_point.json'), 'w', encoding='utf-8') as file:
    json.dump(scatterplot_data, file)